In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img
from tensorflow.keras import backend as K
import keras
from keras.layers import Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, ReLU, Reshape, Dense, Conv2DTranspose, LeakyReLU, Dropout, Flatten
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import Adam

In [2]:
device = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(device[0], True)
device

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Preprocessing

In [12]:
input_shape = (512, 512, 3)
latent_dim = 100
batch_size = 16
data = tf.keras.preprocessing.image_dataset_from_directory(
    "../../../resources/image/paintImg/all_images",
    label_mode=None,
    image_size=(512, 512),
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,

)
data

Found 8027 files belonging to 1 classes.


<BatchDataset element_spec=TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None)>

In [13]:
def normalize(image):
    image = tf.cast(image/255. ,tf.float32)
    return image


In [14]:
data = data.map(normalize)

## Model

# Generator


In [15]:
generator = Sequential()
generator.add(Dense(8 * 8 * 64, input_shape=(latent_dim,), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
generator.add(Reshape((8, 8, 64)))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Conv2DTranspose(256, kernel_size=4, strides=2, padding='same'))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dropout(0.25))

generator.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
generator.add(Conv2DTranspose(128, kernel_size=4, padding='same'))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dropout(0.25))

generator.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
generator.add(Conv2DTranspose(64, kernel_size=4, padding='same'))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Conv2DTranspose(64, kernel_size=4, padding='same'))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Conv2DTranspose(32, kernel_size=4, strides=2, padding='same'))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dropout(0.25))

generator.add(Conv2DTranspose(16, kernel_size=4, strides=2, padding='same'))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Conv2DTranspose(8, kernel_size=4, strides=2, padding='same'))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))

generator.add(Conv2D(3, kernel_size=4, padding='same', activation='sigmoid'))


generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 4096)              413696    
                                                                 
 reshape_1 (Reshape)         (None, 8, 8, 64)          0         
                                                                 
 batch_normalization_11 (Bat  (None, 8, 8, 64)         256       
 chNormalization)                                                
                                                                 
 conv2d_transpose_9 (Conv2DT  (None, 16, 16, 256)      262400    
 ranspose)                                                       
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                                 
 batch_normalization_12 (Bat  (None, 16, 16, 256)     

# Discriminator

In [16]:
discriminator = Sequential()

discriminator.add(Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=(512, 512, 3)))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))

discriminator.add(Conv2D(128, kernel_size=4, strides=2, padding='same'))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))

discriminator.add(Conv2D(256, kernel_size=4, strides=2, padding='same'))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))

discriminator.add(Conv2D(512, kernel_size=4, strides=2, padding='same'))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.25))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 256, 256, 64)      3136      
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 256, 256, 64)      0         
                                                                 
 dropout_10 (Dropout)        (None, 256, 256, 64)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 128, 128, 128)     131200    
                                                                 
 batch_normalization_19 (Bat  (None, 128, 128, 128)    512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 128, 128, 128)     0         
                                                      

# Model

In [17]:
# https://www.kaggle.com/code/karnikakapoor/art-by-gan?scriptVersionId=84113427&cellId=19
class GAN(tf.keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = tf.keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = tf.keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        seed = tf.random.normal(shape=(batch_size, self.latent_dim))
        # Decode them to fake images
        generated_images = self.generator(seed)
        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)
        # Assemble labels discriminating real from fake images
        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))
        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_weights))

        # Sample random points in the latent space
        seed = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(seed))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(), "g_loss": self.g_loss_metric.result()}

In [18]:
epochs = 100
discriminator_opt = Adam(lr=0.0002, beta_1=0.5)
generator_opt = Adam(lr=0.0002, beta_1=0.5)

model = GAN(
    discriminator=discriminator,
    generator=generator,
    latent_dim=latent_dim
)

In [19]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    d_optimizer=discriminator_opt,
    g_optimizer=generator_opt,
    loss_fn=loss_fn
)

callback = keras.callbacks.EarlyStopping(
    monitor='g_loss',
    patience=10
)


In [20]:
history = model.fit(data, epochs=epochs, batch_size=batch_size)

Epoch 1/50
1004/1004 [==============================] - 464s 444ms/step - d_loss: 0.5382 - g_loss: 2.0397
Epoch 2/50
1004/1004 [==============================] - 440s 438ms/step - d_loss: 0.3933 - g_loss: 2.7412
Epoch 3/50
1004/1004 [==============================] - 440s 438ms/step - d_loss: 0.4135 - g_loss: 2.7936
Epoch 4/50
1004/1004 [==============================] - 440s 438ms/step - d_loss: 0.4103 - g_loss: 2.9309
Epoch 5/50
1004/1004 [==============================] - 440s 439ms/step - d_loss: 0.3894 - g_loss: 3.0901
Epoch 6/50
1004/1004 [==============================] - 440s 439ms/step - d_loss: 0.4025 - g_loss: 3.1939
Epoch 7/50
1004/1004 [==============================] - 440s 438ms/step - d_loss: 0.3238 - g_loss: 3.7699
Epoch 8/50
1004/1004 [==============================] - 440s 439ms/step - d_loss: 0.2917 - g_loss: 4.6122
Epoch 9/50
1004/1004 [==============================] - 441s 439ms/step - d_loss: 0.2805 - g_loss: 4.5425
Epoch 10/50
1004/1004 [=======================

 # Display performances curves

In [ ]:
history_dict = history.history
history_dict

In [21]:
g_loss = history_dict['g_loss']
d_loss = history_dict['d_loss']

epochs = range(40)
plt.plot(epochs, d_loss, 'b', label="d_loss")
plt.plot(epochs, g_loss, 'b', label="g_loss", c="red")
plt.title("Loss during training process")
plt.xlabel("Nb epochs")
plt.ylabel("Loss")
plt.yscale('log')
plt.legend()

NameError: name 'history_dict' is not defined

In [22]:
num_img=40

#A function to generate and save images
def Potrait_Generator():
    Generated_Paintings = []
    seed = tf.random.normal([num_img, latent_dim])
    generated_image = generator(seed)
    generated_image *= 255
    generated_image = generated_image.numpy()
    for i in range(num_img):
            img = tf.keras.preprocessing.image.array_to_img(generated_image[i])
            Generated_Paintings.append(img)
            img.save("Potraits{:02d}.png".format(i))
    return

#Generating images
Images = Potrait_Generator()